In [31]:
import scipy.stats as stats
import numpy as np
import math as math
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
def simulate_gbm_time_series(r,q,sigma,T,num_divisons,spot):
    delta_t=T*1.0/num_divisons
    S=spot
    time_series=[]
    time_series.append(S)
    nu=(r-q-(sigma**2/2))
    for i in range(num_divisons):
        z=np.random.standard_normal(1)
        S=S*math.exp(nu*delta_t+sigma*math.sqrt(delta_t)*z)
        time_series.append(S)
    return time_series

def simulate_gbm_time_series_antithetic(r,q,sigma,T,num_divisons,spot):
    delta_t=T*1.0/num_divisons
    S=spot
    Sa=spot
    time_series=[]
    time_series_a=[]
    time_series.append(S)
    time_series_a.append(S)
    nu=(r-q-(sigma**2/2))
    for i in range(num_divisons):
        z=np.random.standard_normal(1)
        S=S*math.exp(nu*delta_t+sigma*math.sqrt(delta_t)*z)
        Sa=Sa*math.exp(nu*delta_t+sigma*math.sqrt(delta_t)*-z)
        time_series.append(S)
        time_series_a.append(Sa)
    return [time_series,time_series_a]

def calculate_price_ci_asian_call(r,q,sigma,T,spot,K,n_simulation_paths,alpha,resampling_rate):
    Y=[]
    price=0
    discouting_factor=math.exp(-r*T) 
    # sampling payoffs
    for i in range(n_simulation_paths):
        time_series=simulate_gbm_time_series(r,q,sigma,T,resampling_rate*20,spot)[::20]
        Y.append(discouting_factor*max(np.mean(time_series)-K,0))

    
    price=np.mean(Y)
    sample_std=np.std(Y)
    
    print("Standard Monte-Carlo variance: "+str(np.var(Y)/n_simulation_paths))

    lower_bound_clt= price-((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    upper_bound_clt= price+((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    
    return (price,lower_bound_clt,upper_bound_clt)



r=0.04
q=0.015
T=1
sigma=0.30
spot=1000
K=1100
m=15
n_simulation_paths=50000
alpha=0.01


print("----Standard Monte-Carlo CI------")
(price,lb,ub)=calculate_price_ci_asian_call(r,q,sigma,T,spot,K,2*n_simulation_paths,alpha,m)
print("Price: "+str(price)+" LB: "+str(lb)+" UB:"+str(ub))


----Standard Monte-Carlo CI------
Standard Monte-Carlo variance: 0.06962878542875957
Price: 35.85029472456419 LB: 35.170603771428496 UB:36.52998567769989


In [33]:
#Using terminal price as the control variate

def pilot_program_terminal_price(r,q,sigma,T,spot,K,n_pilot,alpha,resampling_rate):
    
    Y=[]
    Z=[]
    price=0
    discouting_factor=math.exp(-r*T)
    # sampling payoffs
    for i in range(n_pilot):
        time_series=simulate_gbm_time_series(r,q,sigma,T,resampling_rate*20,spot)[::20] 
        Y.append(discouting_factor*max(np.mean(time_series)-K,0))
        Z.append(time_series[-1])

    Y_bar=np.mean(Y)
    E_Z=spot*math.exp((r-q)*T)
    
    cov_Y_Z=np.sum([(Y[i]-Y_bar)*(Z[i]-E_Z) for i in range(n_pilot)])/(n_pilot-1)
    var_Z=np.sum([(Z[i]-E_Z)**2 for i in range(n_pilot)])/(n_pilot-1)
    
    c=-(cov_Y_Z)/var_Z
    return c
    

def calculate_price_ci_asian_call_terminal_price_control_variate(r,q,sigma,T,spot,K,n_simulation_paths,alpha,resampling_rate):
    V=[]
    price=0
    discouting_factor=math.exp(-r*T)
    c=pilot_program_terminal_price(r,q,sigma,T,spot,K,400,alpha,resampling_rate)
    E_Z=spot*math.exp((r-q)*T)
    # sampling payoffs
    for i in range(n_simulation_paths):
        time_series=simulate_gbm_time_series(r,q,sigma,T,resampling_rate*20,spot)[::20] 
        v= discouting_factor*max(np.mean(time_series)-K,0) + c*(time_series[-1]-E_Z)
        V.append(v)

        
    discouting_factor=math.exp(-r*T) 
    price=np.mean(V)
    sample_std=np.std(V)
    
    print("Control variate: S_T variance: "+str(np.var(V)/n_simulation_paths))

    lower_bound_clt= price-((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    upper_bound_clt= price+((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    
    return (price,lower_bound_clt,upper_bound_clt)


print("---Termial price control variate------")
(price,lb,ub)=calculate_price_ci_asian_call_terminal_price_control_variate(r,q,sigma,T,spot,K,n_simulation_paths,alpha,m)
print("Price: "+str(price)+" LB: "+str(lb)+" UB:"+str(ub))

---Termial price control variate------
Control variate: S_T variance: 0.06722222882334315
Price: 35.94515702075569 LB: 35.27731531980174 UB:36.61299872170964


In [34]:
def black_scholes (cp, s, k, t, v, rf, div):
        """ Price an option using the Black-Scholes model.
        s: initial stock price
        k: strike price
        t: expiration time
        v: volatility
        rf: risk-free rate
        div: dividend
        cp: +1/-1 for call/put
        """

        d1 = (math.log(s/k)+(rf-div+0.5*math.pow(v,2))*t)/(v*math.sqrt(t))
        d2 = d1 - v*math.sqrt(t)

        optprice = (cp*s*math.exp(-div*t)*stats.norm.cdf(cp*d1)) - (cp*k*math.exp(-rf*t)*stats.norm.cdf(cp*d2))
        return optprice


def pilot_program_bs_price(r,q,sigma,T,spot,K,n_pilot,alpha,resampling_rate):
    
    Y=[]
    Z=[]
    price=0
    discouting_factor=math.exp(-r*T) 
    # sampling payoffs
    for i in range(n_pilot):
        time_series=simulate_gbm_time_series(r,q,sigma,T,resampling_rate*20,spot)[::20] 
        Y.append(discouting_factor*max(np.mean(time_series)-K,0))
        Z.append(discouting_factor*max(time_series[-1]-K,0))

    Y_bar=np.mean(Y)
    E_Z=black_scholes(1, spot, K, T, sigma, r, q)
    
    cov_Y_Z=np.sum([(Y[i]-Y_bar)*(Z[i]-E_Z) for i in range(n_pilot)])/(n_pilot-1)
    var_Z=np.sum([(Z[i]-E_Z)**2 for i in range(n_pilot)])/(n_pilot-1)
    
    c=-(cov_Y_Z)/var_Z
    return c
    

def calculate_price_ci_asian_call_bs_price_control_variate(r,q,sigma,T,spot,K,n_simulation_paths,alpha,resampling_rate):
    V=[]
    price=0
    c=pilot_program_bs_price(r,q,sigma,T,spot,K,400,alpha,resampling_rate)
    discouting_factor=math.exp(-r*T) 
    E_Z=black_scholes(1, spot, K, T, sigma, r, q)
    # sampling payoffs
    for i in range(n_simulation_paths):
        time_series=simulate_gbm_time_series(r,q,sigma,T,resampling_rate*20,spot)[::20] 
        v= discouting_factor*max(np.mean(time_series)-K,0) + c*(discouting_factor*max(time_series[-1]-K,0)-E_Z)
        V.append(v)

    price=np.mean(V)
    sample_std=np.std(V)
    
    print("Control variate: Black Scholes price variance: "+str(np.var(V)/n_simulation_paths))

    lower_bound_clt= price-((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    upper_bound_clt= price+((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    
    return (price,lower_bound_clt,upper_bound_clt)


print("---Black Scholes control variate------")
(price,lb,ub)=calculate_price_ci_asian_call_bs_price_control_variate(r,q,sigma,T,spot,K,n_simulation_paths,alpha,m)
print("Price: "+str(price)+" LB: "+str(lb)+" UB:"+str(ub))

---Black Scholes control variate------
Control variate: Black Scholes price variance: 0.04340471172604791
Price: 35.60624568981527 LB: 35.069602629392925 UB:36.14288875023761


In [39]:
def black_scholes_geo_asian_option (cp, s, k, t, v, rf, div):
        """ Price an option using the Black-Scholes model.
        s: initial stock price
        k: strike price
        t: expiration time
        v: volatility
        rf: risk-free rate
        div: dividend
        cp: +1/-1 for call/put
        """
        nu=(r-q-(v**2/2))
        d1 = (t*math.log(s/k)+(nu*t*t/2)+v*v*t*t/3)/(v*math.sqrt(t*t*t/3))
        d2 = (t*math.log(s/k)+(nu*t*t/2))/(v*math.sqrt(t*t*t/3))

        optprice = (s*math.exp(-nu*t/2)*stats.norm.cdf(cp*d1)) - (cp*k*stats.norm.cdf(cp*d2))
        return math.exp(-rf*t)*optprice


def pilot_program_geo_price(r,q,sigma,T,spot,K,n_pilot,alpha,resampling_rate):
    
    Y=[]
    Z=[]
    price=0
    discouting_factor=math.exp(-r*T) 
    # sampling payoffs
    for i in range(n_pilot):
        time_series=simulate_gbm_time_series(r,q,sigma,T,resampling_rate*20,spot)[::20] 
        Y.append(discouting_factor*max(np.mean(time_series)-K,0))
        Z.append(discouting_factor*max(stats.mstats.gmean(time_series)-K,0))

    Y_bar=np.mean(Y)
    E_Z=32.14
    #E_Z=black_scholes_geo_asian_option(1, spot, K, T, sigma, r, q)
    
    cov_Y_Z=np.sum([(Y[i]-Y_bar)*(Z[i]-E_Z) for i in range(n_pilot)])/(n_pilot-1)
    var_Z=np.sum([(Z[i]-E_Z)**2 for i in range(n_pilot)])/(n_pilot-1)
    
    c=-(cov_Y_Z)/var_Z
    return c
    

def calculate_price_ci_asian_call_geo_asian_control_variate(r,q,sigma,T,spot,K,n_simulation_paths,alpha,resampling_rate):
    V=[]
    price=0
    c=pilot_program_geo_price(r,q,sigma,T,spot,K,400,alpha,resampling_rate)
    discouting_factor=math.exp(-r*T) 
    E_Z=32.14
    #E_Z=black_scholes_geo_asian_option(1, spot, K, T, sigma, r, q)
    # sampling payoffs
    for i in range(n_simulation_paths):
        time_series=simulate_gbm_time_series(r,q,sigma,T,resampling_rate*20,spot)[::20] 
        v= discouting_factor*max(np.mean(time_series)-K,0) + c*(discouting_factor*max(stats.mstats.gmean(time_series)-K,0)-E_Z)
        V.append(v)

    price=np.mean(V)
    sample_std=np.std(V)
    
    print("Control variate: Geometric Asian Option price variance: "+str(np.var(V)/n_simulation_paths))

    lower_bound_clt= price-((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    upper_bound_clt= price+((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    
    return (price,lower_bound_clt,upper_bound_clt)


print("---Geometric Asian Option control variate------")
(price,lb,ub)=calculate_price_ci_asian_call_geo_asian_control_variate(r,q,sigma,T,spot,K,n_simulation_paths,alpha,m)
print("Price: "+str(price)+" LB: "+str(lb)+" UB:"+str(ub))

---Geometric Asian Option control variate------
Control variate: Geometric Asian Option price variance: 0.0004130187649946996
Price: 35.50985841952891 LB: 35.45751019319772 UB:35.5622066458601


In [36]:
black_scholes_geo_asian_option(1, spot, K, T, sigma, r, q)


35.008981260989806